In [28]:
import pandas as pd
import json
from datetime import datetime, timedelta
import random

In [29]:
csv_file_path = 'TabelaNutrIA-Tratada.csv'
df = pd.read_csv(csv_file_path, sep=',')
jsonl_file_path = 'output-backend.jsonl'

In [30]:
dates = {
    'hoje': datetime.now().strftime('%Y-%m-%d'),
    'ontem': (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'),
    'amanha': (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
}

datas_chaves = list(dates.keys())
valores_porcao_padrao = [1, 50, 100, 200]
valores_quantidades = [1, 25, 30, 40, 50, 55, 75, 60, 93, 100]


In [31]:
content = f"Você é um assistente nutricional especializado e sua função é fornecer respostas claras e diretas sobre tópicos de nutrição, incluindo planejamento de refeições, controle de calorias e dicas para uma alimentação saudável baseados nos pilares de dietas flexível, contagem de macronutrientes e balanço energético. Você deve invocar as funções quando necessário."

### Resposta Completa

In [32]:
with open(jsonl_file_path, "w", encoding="utf-8") as jsonl_file:
    for index, row in df.head(1000).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = f"Lembrando que hoje é dia {dates['hoje']}. Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} na minha refeicao {numero_refeicao} de {data_chave}."
        arguments = {
            "alimentos": [{
                    "nome_consumo":row["nome_alimento"],
                    "numero_refeicao":numero_refeicao,
                    "id_alimento": None,
                    "id_prato":None,
                    "dt_dia":dates[data_chave],
                    "unidade_medida":"GRAMA",
                    "porcao_padrao": porcao_padrao,
                    "qtde_utilizada":qtde_utilizada_porcao,
                    "qtde_proteina":round(fator_multiplicador * row["qtde_proteina"], 1),
                    "qtde_carboidrato":round(
                        fator_multiplicador * row["qtde_carboidrato"], 1
                    ),
                    "qtde_gordura":round(fator_multiplicador * row["qtde_gordura"], 1),
                    "qtde_alcool":round(fator_multiplicador * row["qtde_alcool"], 1),
                    "kcal":round(fator_multiplicador * row["kcal"], 1),
                }]
         }

        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments),
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {   "role": "user", 
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [resposta_assistente],
                },
            ]
        }

        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem refeição

In [33]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(100).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = f"Lembrando que hoje é dia {dates['hoje']}. Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} {data_chave}."

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Por favor informe o número da refeição.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem alimento

In [34]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(50).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = f"Lembrando que hoje é dia {dates['hoje']}. Adicionar {numero_refeicao} de {data_chave}."

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Me informe o nome do alimento por favor.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + '\n')

print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem Quantidade

In [35]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(100).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = f"Lembrando que hoje é dia {dates['hoje']}. Adicionar {row['nome_alimento']} na minha refeicao {numero_refeicao} de {data_chave}."

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Por favor informe a quantidade.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + '\n')

print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


# Assumir data Atual sem infor

In [36]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:
    
    for index, row in df.head(150).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        porcao_padrao = 1
        qtde_utilizada_porcao = random.choice(valores_quantidades)
        qtde_utilizada_gramas = qtde_utilizada_porcao * porcao_padrao
        fator_multiplicador = qtde_utilizada_gramas / row['porcao_padrao']
        # qtde_utilizada = round(qtde_utilizada_gramas / porcao_padrao, 1)
        entrada_usuario = f"Lembrando que hoje é dia {dates['hoje']}. Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} {row['estado_alimento']} na minha refeicao {numero_refeicao}."

        arguments = {
            "alimentos": [{
                    "numero_refeicao":numero_refeicao,
                    "id_alimento": None,
                    "id_prato":None,
                    "dt_dia":dates['hoje'],
                    "unidade_medida":"GRAMA",
                    "porcao_padrao": porcao_padrao,
                    "qtde_utilizada":qtde_utilizada_porcao,
                    "qtde_proteina":round(fator_multiplicador * row["qtde_proteina"], 1),
                    "qtde_carboidrato":round(
                        fator_multiplicador * row["qtde_carboidrato"], 1
                    ),
                    "qtde_gordura":round(fator_multiplicador * row["qtde_gordura"], 1),
                    "qtde_alcool":round(fator_multiplicador * row["qtde_alcool"], 1),
                    "kcal":round(fator_multiplicador * row["kcal"], 1),
                }]
        }

        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments)
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {"role": "user", "content": entrada_usuario},
                {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [resposta_assistente],
                },
            ]
        }

        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.
